In [1]:
model_name="intfloat/multilingual-e5-large-instruct"

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [3]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 96.8MB/s]


In [4]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [5]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 33.0 MB/s eta 0:00:00
   ━━━━

In [6]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [7]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [8]:


models_list_3=[
        model_name,
          ]



models_name_3=[
        model_name,

]

In [892]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["som"]
# langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [893]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [894]:
train_data=all_csv


In [895]:
train_data=train_data.fillna(0)

In [896]:
float_columns = train_data.select_dtypes(include=['float64']).columns
# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')

In [897]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,som_train_track_a_00001,0,0,0,0,1,0,som,ilahay hakuuugargaaaro wlllo wlhi waaanlaoooya...
1,1,som_train_track_a_00002,0,0,0,0,0,0,som,mahadsanid walaaalkiiis lkn iniga ugu yaraaan ...
2,2,som_train_track_a_00003,0,0,0,1,0,0,som,hambalyo ummmaddda sooomaaaliyeeed waxaaan laq...
3,3,som_train_track_a_00004,0,0,0,1,0,0,som,waxa uuu ahaaa faqiiir lkn jacayl ayaaa hayay ...
4,4,som_train_track_a_00005,0,0,0,0,1,0,som,iskal ilaaasha sinada iyo siiigada xuman allla...
...,...,...,...,...,...,...,...,...,...,...
3387,3387,som_train_track_a_03388,0,0,0,0,1,0,som,hadaaan sidiii futo ari looo feydin ma fayooob...
3388,3388,som_train_track_a_03389,0,1,0,0,0,0,som,allla maxa barooor kadhacaysa qoladiii
3389,3389,som_train_track_a_03390,1,0,0,0,0,0,som,garaaad jaaamac garaaad ali garaaadka guuud ee...
3390,3390,som_train_track_a_03391,0,0,0,0,0,0,som,taliska ciiidanka xoooggga dalka ooo war kasoo...


In [898]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness']


In [899]:
train_data['lang'].value_counts()

,count
lang,
som,3392


In [900]:
sum(train_data['lang'].isna())

0

In [901]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [902]:
train_data['clean_message']=train_data['clean_message'].astype(str)

In [903]:
maxi=train_data['clean_message'].apply(len).max()
maxi

645

In [904]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
2600,som_train_track_a_02601,0,1,0,0,0,0,som,ma noli hhh cunugay ika so dhacaygrinning face...


In [905]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['ma noli hhh cunugay ika so dhacaygrinning face with big eyes face with tears of joy rolling on the floor laughing face with tears of joy grinning face with big eyes face with tears of joy face with tears of joy face with tears of joy tihan dofargrinning face with big eyes face with tears of joy rolling on the floor laughing rolling on the floor laughing rolling on the floor laughing face with tears of joy grinning face with big eyes grinning face with big eyes grinning face with big eyes face with tears of joy face with tears of joy face with tears of joy grinning face with big eyes grinning face with big eyes grinning face with big eyes'],
      dtype=object)

In [906]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [907]:
# train valid split
# train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [908]:
x_train, y_train = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [909]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [910]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        3064      328  3392
disgust      2915      477  3392
fear         3087      305  3392
joy          2797      595  3392
sadness      3001      391  3392
surprise     3213      179  3392


In [911]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [912]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([5.1707, 3.5556, 5.5607, 2.8504, 4.3376, 9.4749], device='cuda:0')

In [913]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [914]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [915]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [916]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [917]:
USE_LORA=False

In [918]:
!pip install -q sentence_transformers

In [919]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [920]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
# embedding_model = SentenceTransformer(
#         x,
#         trust_remote_code=True,
#         device="cpu" if not use_gpu else "cuda",
#         model_kwargs={
#             "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
#             # "attn_implementation": "sdpa"
#         },
#         # vector_dim must be 12288, 1024, 512, 256
#         ## 1024 is recommended
#         # set is_text_encoder 'True', if you do not encode image
#         # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
#     )
#     # We can reduce the max_seq_length from the default of 2048 for faster encoding
# embedding_model.max_seq_length = MAX_LENGTH

intfloat/multilingual-e5-large-instruct


In [921]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [922]:
# a=

In [923]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [924]:
device

'cuda'

In [925]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [926]:

# Custom Dataset for batching
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [927]:
from torch.utils.data import DataLoader

In [928]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [929]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 106/106 [01:35<00:00,  1.12it/s]


In [930]:
np.save("x_train_embeddings.npy",x_train_embeddings)

In [931]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)

In [932]:
import xgboost as xgb

In [933]:
# xgb_models = {}

# for i, label in enumerate(label_columns):
#     print(f"Training XGBoost for label: {label}")
#     # Use the computed class weights for the positive class
#     scale_pos_weight = class_weights_tensor[i].item()
#     # Initialize and train XGBoost classifier
#     xgb_model = xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='logloss',
#         use_label_encoder=False,
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=6,
#         tree_method='hist',
#         scale_pos_weight=scale_pos_weight,  # Apply the class weight here
#         device=device
#     )
#     xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
#     xgb_models[label] = xgb_model

In [934]:
# Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     model.save_model(f"xgb_model_{label}.json")

In [935]:
# import pickle
# # Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     with open(f"xgb_model_{label}.pkl", 'wb') as f:
#       pickle.dump(model, f)

In [936]:
# !zip xgb_models_pkl.zip xgb_model_*.pkl

In [937]:
!rm -rf svm*

In [938]:
from sklearn.svm import SVC

In [939]:
svm_models = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

Training SVM for label: anger
Training SVM for label: disgust
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [940]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [941]:
!zip svm_models_pkl.zip svm_model_*.pkl

  adding: svm_model_anger.pkl (deflated 75%)
  adding: svm_model_disgust.pkl (deflated 75%)
  adding: svm_model_fear.pkl (deflated 75%)
  adding: svm_model_joy.pkl (deflated 75%)
  adding: svm_model_sadness.pkl (deflated 75%)
  adding: svm_model_surprise.pkl (deflated 75%)


In [942]:
kind="dev"

In [943]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)
dev_data=all_csv
dev_data=dev_data.fillna(0)
float_columns = dev_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
dev_data[float_columns] = dev_data[float_columns].astype('int')
dev_data = dev_data[dev_data['clean_message'].isnull()==False]
dev_data.reset_index()


,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,som_dev_track_a_00001,0,0,0,0,0,0,som,albaaab walba ooo kaaa xiran alaha kuuu furo m...
1,1,som_dev_track_a_00002,0,0,0,0,0,0,som,wadan fashilmay somalia alllow dalkayga somali...
2,2,som_dev_track_a_00003,0,0,0,0,0,0,som,aduuunkan farxad mataaalo cos adigooooooyneya ...
3,3,som_dev_track_a_00004,0,0,0,0,0,0,som,maro madow ina sooo xiratan bey aheyd wa cabsi
4,4,som_dev_track_a_00005,0,0,0,0,0,0,som,meshan waxabaaakadhacay malintaaan qurux ku dh...
...,...,...,...,...,...,...,...,...,...,...
561,561,som_dev_track_a_00562,0,0,0,0,0,0,som,rwanda waxa ka dhacay 19994 dagaaal sokeeeye o...
562,562,som_dev_track_a_00563,0,0,0,0,0,0,som,wlhi wlll wa runta inanku cawaaan ayu ahaaa ha...
563,563,som_dev_track_a_00564,0,0,0,0,0,0,som,welcome to mugadisho gacan furan ku sooo dhawa...
564,564,som_dev_track_a_00565,0,0,0,0,0,0,som,user uae ooo dhistay djibouti deladeeeda kadib...


In [944]:
x_dev, y_dev = dev_data['clean_message'].values.tolist(), dev_data[label_columns].values.tolist()

In [945]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [946]:
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 18/18 [00:14<00:00,  1.25it/s]


In [947]:
y_dev_flat = np.array(y_dev)

In [948]:
# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = svm_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: disgust
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


In [949]:
predictions_df = pd.DataFrame(test_predictions)
dev_data.reset_index(inplace=True)

In [950]:
output_df = pd.concat([dev_data[['id','lang']], predictions_df], axis=1)

In [951]:
!rm -rf track*

In [952]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'som' saved to track_a/pred_som.csv


In [953]:
!zip -r {task}.zip {task}

  adding: track_a/ (stored 0%)
  adding: track_a/pred_som.csv (deflated 90%)
